## Analisando e tratando os dados do arquivp Person.Person.csv que estão na pasta RAW
### Ao final desse notebook os dados serão carregados na pasta REFINED em formato parquet

In [1]:
# importando as bibliotecas necessárias
import pandas as pd
import numpy as np

In [2]:
# criando a variavel com o caminho do arquivo que iremos carregar
path_person = "gs://bike-factory-datalake/01.RAW/Person.Person.csv"

In [3]:
# carregando o arquivo CSV
person = pd.read_csv(path_person,sep=';')

In [4]:
# criando o DF pandas
df_person = pd.DataFrame(person)

In [5]:
df_person.dtypes

BusinessEntityID          int64
PersonType               object
NameStyle                 int64
Title                    object
FirstName                object
MiddleName               object
LastName                 object
Suffix                   object
EmailPromotion            int64
AdditionalContactInfo    object
Demographics             object
rowguid                  object
ModifiedDate             object
dtype: object

In [6]:
# verificando as primeiras linhas do DF
df_person.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,0,NaN,Ken,J,Sánchez,NaN,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07 00:00:00.000
1,2,EM,0,NaN,Terri,Lee,Duffy,NaN,1,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24 00:00:00.000
2,3,EM,0,NaN,Roberto,NaN,Tamburello,NaN,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04 00:00:00.000
3,4,EM,0,NaN,Rob,NaN,Walters,NaN,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28 00:00:00.000
4,5,EM,0,Ms.,Gail,A,Erickson,NaN,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30 00:00:00.000


In [7]:
# Coletando informações básicas do DF
# A saída desse comando será a quantidade de linhas e colunas (linhas, colunas)
df_person.shape

(19972, 13)

In [8]:
# verificando o index do arquivo
df_person.index

RangeIndex(start=0, stop=19972, step=1)

In [9]:
# Verificando as colunas do DF
df_person.columns

Index(['BusinessEntityID', 'PersonType', 'NameStyle', 'Title', 'FirstName',
       'MiddleName', 'LastName', 'Suffix', 'EmailPromotion',
       'AdditionalContactInfo', 'Demographics', 'rowguid', 'ModifiedDate'],
      dtype='object')

In [10]:
# Contagem de dados não nulos
df_person.count()

BusinessEntityID         19972
PersonType               19972
NameStyle                19972
Title                     1009
FirstName                19972
MiddleName               11473
LastName                 19972
Suffix                      53
EmailPromotion           19972
AdditionalContactInfo       10
Demographics             19972
rowguid                  19972
ModifiedDate             19972
dtype: int64

In [11]:
# Identificando a quantidade de dados nulos em cada coluna
df_person.isnull().sum()

BusinessEntityID             0
PersonType                   0
NameStyle                    0
Title                    18963
FirstName                    0
MiddleName                8499
LastName                     0
Suffix                   19919
EmailPromotion               0
AdditionalContactInfo    19962
Demographics                 0
rowguid                      0
ModifiedDate                 0
dtype: int64

Até aqui sabemos que a coluna Tittle possui 18.963 campos nulos, MiddleName 8.499, Suffix 19.919 e AdditionalContactInfo 19.962.

Temos que avaliar se essas colunas com dados faltantes são essenciais ou não para nossa análise, se não forem essenciais podemos dropar as linhas com dados nulos ou as colunas e se forem essencias, devemos pensar em uma maneira de substituir esses valores.

Por exemplo, a coluna Title não faz diferença na nossa análise, porém sem droparmos as linhas com dados nulos, ficarão apenas 1.009 registros e não queremos isso, então nesse caso droparemos a coluna para não levar os dados nulos para a próxima camada do nosso Lake.

In [12]:
# Dropando ca coluna 'Title'
df_person = df_person.drop('Title',axis=1)

In [13]:
df_person.isnull().sum()

BusinessEntityID             0
PersonType                   0
NameStyle                    0
FirstName                    0
MiddleName                8499
LastName                     0
Suffix                   19919
EmailPromotion               0
AdditionalContactInfo    19962
Demographics                 0
rowguid                      0
ModifiedDate                 0
dtype: int64

Avaliando as colunas MiddleName e Suffix, vemos que elas fazem parte do nome completo do cliente, então aqui podemos contatenar todas as colunas que possuem relação com o nome em uma única coluna com o CompleteName

In [14]:
# Antes temos que converter os valores NaN que estão nas colunas em uma string vazia
df_person.MiddleName = df_person.MiddleName.fillna('')
df_person.Suffix = df_person.Suffix.fillna('')

In [15]:
df_person.head()

,BusinessEntityID,PersonType,NameStyle,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,0,Ken,J,Sánchez,,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07 00:00:00.000
1,2,EM,0,Terri,Lee,Duffy,,1,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24 00:00:00.000
2,3,EM,0,Roberto,,Tamburello,,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04 00:00:00.000
3,4,EM,0,Rob,,Walters,,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28 00:00:00.000
4,5,EM,0,Gail,A,Erickson,,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30 00:00:00.000


In [16]:
df_person.isnull().sum()

BusinessEntityID             0
PersonType                   0
NameStyle                    0
FirstName                    0
MiddleName                   0
LastName                     0
Suffix                       0
EmailPromotion               0
AdditionalContactInfo    19962
Demographics                 0
rowguid                      0
ModifiedDate                 0
dtype: int64

In [17]:
# Contatenando as colunas FirstName, MiddleName, LastName e Suffix para criar a coluna CompleteName
df_person['CompleteName'] = df_person['FirstName'].map(str) + ' ' + df_person['MiddleName'].map(str) + ' ' + df_person['LastName'].map(str) + ' ' + df_person['Suffix'].map(str)   

In [18]:
# Dropando as colunas que não precisamos mais
df_person = df_person.drop(['FirstName','MiddleName','LastName','Suffix'], axis = 1)

In [19]:
df_person.head()

,BusinessEntityID,PersonType,NameStyle,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate,CompleteName
0,1,EM,0,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07 00:00:00.000,Ken J Sánchez
1,2,EM,0,1,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24 00:00:00.000,Terri Lee Duffy
2,3,EM,0,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04 00:00:00.000,Roberto Tamburello
3,4,EM,0,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28 00:00:00.000,Rob Walters
4,5,EM,0,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30 00:00:00.000,Gail A Erickson


Temos também a coluna AdditionalContactInfo que só possuem 10 registros válidos, vamos tomar para esse caso que ela não é necessária devido ao problema de negócio que temos que responder, portanto ela será dropada

In [20]:
# Dropando a coluna AdditionalContactInfo
df_person = df_person.drop('AdditionalContactInfo',axis=1)

Ainda falta tratar a coluna ModifiedDate que não está com o formato de data

In [21]:
df_person['ModifiedDate']= pd.to_datetime(df_person['ModifiedDate'])

In [22]:
# Confirmando se a coluna recebeu tipo correto
df_person.dtypes

BusinessEntityID             int64
PersonType                  object
NameStyle                    int64
EmailPromotion               int64
Demographics                object
rowguid                     object
ModifiedDate        datetime64[ns]
CompleteName                object
dtype: object

Por fim, já temos todas as nossas colunas tratadas, então iremos salvar o DF em um arquivo parquet que é otimizado para o modelo analítico

In [23]:
df_person.to_parquet('gs://bike-factory-datalake/02.REFINED/person.parquet')